In [4]:
%run Functions.ipynb
%run Initialiser.ipynb
%run plotinfo.ipynb

#importing required libraries
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
from bokeh.models import Arrow, NormalHead
from bokeh.palettes import Category10
from bokeh.plotting import figure, show
from bokeh.plotting import row
from bokeh.plotting import column
from bokeh.models import Legend
from bokeh.io import output_file
from bokeh.models import ColumnDataSource, TableColumn, DataTable, NumberFormatter, Div
from bokeh.layouts import layout
from bokeh.plotting import curdoc
import copy as copy
#________________________________________________________________________________________________#
if fluid!="water":
    for i in range(0,numberofstates):
        if (componenttype[i]=="Compressor" or componenttype[i]=="compressor" or componenttype[i]=="turbine" or componenttype[i]=="Turbine") and processtype[i]=="":
            processtype[i]="adiabatic"
        if (componenttype[i]=="Condensor" or componenttype[i]=="condensor" or componenttype[i]=="Boiler" or componenttype[i]=="boiler") and processtype[i]=="":
            processtype[i]="isobaric"
        if (componenttype[i]=="Throttle" or componenttype[i]=="throttle"):
            processtype[i]="isenthalpic"
        if processtype[i]=="isenthalpic" or processtype[i]=="Isenthalpic":
            componenttype[i]="throttle"

    for i in range(0,numberofstates):
        getallgasproperties(propslist,i+1) #first getting all possible properties from given data.
        if Tratio[i]!=0:
            if i==numberofstates-1: #i.e last value in the list.
                if T[i]!=0 and T[0]==0:
                    T[0]=T[i]*Tratio[i]
                elif T[i]==0 and T[0]!=0:
                    T[i]=T[0]/Tratio[i]
            else:
                if T[i]!=0 and T[i+1]==0: #if T[i] has been mentioned but T[i+1] hasnt been mentioned.
                    T[i+1]=T[i]*Tratio[i]
                elif T[i+1]!=0 and T[i]==0:
                    T[i]=T[i+1]/Tratio[i]
        if Pratio[i]!=0:
            if i==numberofstates-1: #i.e last value in the list.
                if P[i]!=0 and P[0]==0:
                    P[0]=P[i]*Pratio[i]
                elif P[i]==0 and P[0]!=0:
                    P[i]=P[0]/Pratio[i]
            else:
                if P[i]!=0 and P[i+1]==0: 
                    P[i+1]=P[i]*Pratio[i]
                elif P[i+1]!=0 and P[i]==0:
                    P[i]=P[i+1]/Pratio[i]
        if vratio[i]!=0:
            if i==numberofstates-1: #i.e last value in the list.
                if v[i]!=0 and v[0]==0:
                    v[0]=v[i]*vratio[i]
                elif v[i]==0 and v[0]!=0:
                    v[i]=v[0]/vratio[i]
            else:
                if v[i]!=0 and v[i+1]==0: 
                    v[i+1]=v[i]*vratio[i]
                elif v[i+1]!=0 and v[i]==0:
                    v[i]=v[i+1]/vratio[i]

    for i in range(numberofstates):
        if processtype[i]=="isobaric" or processtype[i]=="Isobaric": #P constant
            wtransfer[i]=0
            if i==numberofstates-1:
                if P[i]==0 and P[0]!=0:
                    P[i]=P[0]
                elif P[i]!=0 and P[0]==0:
                    P[0]=P[i]
            else:
                if P[i+1]==0 and P[i]!=0:
                    P[i+1]=P[i]
                elif P[i+1]!=0 and P[i]==0:
                    P[i]=P[i+1]
        getallgasproperties(propslist,i+1)

        if processtype[i]=="isothermal" or processtype[i]=="Isothermal": #T constant
            if i==numberofstates-1:
                if T[i]==0 and T[0]!=0:
                    T[i]=T[0]
                elif T[i]!=0 and T[0]==0:
                    T[0]=T[i]
            else:
                if T[i+1]==0 and T[i]!=0:
                    T[i+1]=T[i]
                elif T[i+1]!=0 and T[i]==0:
                    T[i]=T[i+1]
        getallgasproperties(propslist,i+1)

        if processtype[i]=="isochoric" or processtype[i]=="Isochoric":
            if i==numberofstates-1:
                if v[i]==0 and v[0]!=0:
                    v[i]=v[0]
                elif v[i]!=0 and v[0]==0:
                    v[0]=v[i]
            else:
                if v[i+1]==0 and v[i]!=0:
                    v[i+1]=v[i]
                elif v[i+1]!=0 and v[i]==0:
                    v[i]=v[i+1]
        getallgasproperties(propslist,i+1)

        if processtype[i]=="isenthalpic" or processtype[i]=="Isenthalpic":
            qtransfer[i]=0
            wtransfer[i]=0
            Qdot[i]=0
            Wdot[i]=0
            if i==numberofstates-1:
                if h[i]==0 and h[0]!=0:
                    h[i]=h[0]
                elif h[i]!=0 and h[0]==0:
                    h[0]=h[i]
            else:
                if h[i+1]==0 and h[i]!=0:
                    h[i+1]=h[i]
                elif h[i+1]!=0 and h[i]==0:
                    h[i]=h[i+1]
        getallgasproperties(propslist,i+1)

        if (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]==1:
            qtransfer[i]=0
            if i==numberofstates-1:
                if s[0]!=0 and s[i]==0:
                    s[i]=s[0]
                elif s[0]==0 and s[i]!=0:
                    s[0]=s[i]
            else:
                if s[i+1]!=0 and s[i]==0:
                    s[i]=s[i+1]
                elif s[i]!=0 and s[i+1]==0:
                    s[i+1]=s[i] 
        getallgasproperties(propslist,i+1)

    for i in range(numberofstates):
        if (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]==1:
            qtransfer[i]=0
            if i==numberofstates-1:
                if s[0]!=0 and s[i]==0:
                    s[i]=s[0]
                elif s[0]==0 and s[i]!=0:
                    s[0]=s[i]
            else:
                if s[i+1]!=0 and s[i]==0:
                    s[i]=s[i+1]
                elif s[i]!=0 and s[i+1]==0:
                    s[i+1]=s[i] 

    #test
    if (processtype[0]=="Adiabatic" or processtype[0]=="adiabatic") and isenefficiency[0]==1:
        s[1]=s[0]

    for i in range(numberofstates):
        if qtransfer[i]!="" and Qdot[i]!="" and qtransfer[i]!=0 and Qdot[1]!=0 and massflowrate==0: #i.e massflowrate not given, must be solved for.
            massflowrate=(Qdot[i])/(qtransfer[i])
        if wtransfer[i]!="" and Wdot[i]!="" and wtransfer[i]!=0 and Wdot[1]!=0 and massflowrate==0: #i.e massflowrate not given, must be solved for.
            massflowrate=(Wdot[i])/(wtransfer[i])

    for i in range(numberofstates):
        if qtransfer[i]=="" and Qdot[i]!="" and massflowrate!=0:
            qtransfer[i]=(Qdot[i])/massflowrate
            
        if Qdot[i]=="" and qtransfer[i]!="" and massflowrate!=0:
            Qdot[i]=massflowrate*qtransfer[i]
            
        if wtransfer[i]=="" and Wdot[i]!="" and massflowrate!=0:
            wtransfer[i]=(Wdot[i])/massflowrate
            
        if Wdot[i]=="" and wtransfer[i]!="" and massflowrate!=0:
            Wdot[i]=massflowrate*wtransfer[i]

    #----------------------------------------------------------------------------------------------------------------------------#
    #next, need to do SFEE.

    #again updating the list:
    for i in range(0,numberofstates):
        getallgasproperties(propslist,i+1)

    for i in range(numberofstates):
        if processtype[i]=="isobaric" or processtype[i]=="Isobaric":
            wtransfer[i]=0
        if processtype[i]=="adiabatic" or processtype[i]=="Adiabatic":
            qtransfer[i]=0

        
        #added here too, just in case

    #MAIN LOOP
    for a in range(0,30):
        for i in range(numberofstates):
            if qtransfer[i]!="" and Qdot[i]!="" and qtransfer[i]!=0 and Qdot[1]!=0 and massflowrate==0: #i.e massflowrate not given, must be solved for.
                massflowrate=(Qdot[i])/(qtransfer[i])
            if wtransfer[i]!="" and Wdot[i]!="" and wtransfer[i]!=0 and Wdot[1]!=0 and massflowrate==0: #i.e massflowrate not given, must be solved for.
                massflowrate=(Wdot[i])/(wtransfer[i])

        for i in range(numberofstates):
            if qtransfer[i]=="" and Qdot[i]!="" and massflowrate!=0:
                qtransfer[i]=(Qdot[i])/massflowrate
                
            if Qdot[i]=="" and qtransfer[i]!="" and massflowrate!=0:
                Qdot[i]=massflowrate*qtransfer[i]
                
            if wtransfer[i]=="" and Wdot[i]!="" and massflowrate!=0:
                wtransfer[i]=(Wdot[i])/massflowrate
                
            if Wdot[i]=="" and wtransfer[i]!="" and massflowrate!=0:
                Wdot[i]=massflowrate*wtransfer[i]
            
            getallgasproperties(propslist,i+1)
            
        for i in range(numberofstates):
            getallgasproperties(propslist,i+1) #first getting all possible properties from given data.
            if Tratio[i]!=0:
                if i==numberofstates-1: #i.e last value in the list.
                    if T[i]!=0 and T[0]==0:
                        T[0]=T[i]*Tratio[i]
                    elif T[i]==0 and T[0]!=0:
                        T[i]=T[0]/Tratio[i]
                else:
                    if T[i]!=0 and T[i+1]==0: #if T[i] has been mentioned but T[i+1] hasnt been mentioned.
                        T[i+1]=T[i]*Tratio[i]
                    elif T[i+1]!=0 and T[i]==0:
                        T[i]=T[i+1]/Tratio[i]
            if Pratio[i]!=0:
                if i==numberofstates-1: #i.e last value in the list.
                    if P[i]!=0 and P[0]==0:
                        P[0]=P[i]*Pratio[i]
                    elif P[i]==0 and P[0]!=0:
                        P[i]=P[0]/Pratio[i]
                else:
                    if P[i]!=0 and P[i+1]==0: 
                        P[i+1]=P[i]*Pratio[i]
                    elif P[i+1]!=0 and P[i]==0:
                        P[i]=P[i+1]/Pratio[i]
            if vratio[i]!=0:
                if i==numberofstates-1: #i.e last value in the list.
                    if v[i]!=0 and v[0]==0:
                        v[0]=v[i]*vratio[i]
                    elif v[i]==0 and v[0]!=0:
                        v[i]=v[0]/vratio[i]
                else:
                    if v[i]!=0 and v[i+1]==0: 
                        v[i+1]=v[i]*vratio[i]
                    elif v[i+1]!=0 and v[i]==0:
                        v[i]=v[i+1]/vratio[i]


                        
            if processtype[i]=="isobaric" or processtype[i]=="Isobaric": #P constant
                wtransfer[i]=0
                if i==numberofstates-1:
                    if P[i]==0 and P[0]!=0:
                        P[i]=P[0]
                    elif P[i]!=0 and P[0]==0:
                        P[0]=P[i]
                else:
                    if P[i+1]==0 and P[i]!=0:
                        P[i+1]=P[i]
                    elif P[i+1]!=0 and P[i]==0:
                        P[i]=P[i+1]
            getallgasproperties(propslist,i+1)

            if processtype[i]=="isothermal" or processtype[i]=="Isothermal": #T constant
                if i==numberofstates-1:
                    if T[i]==0 and T[0]!=0:
                        T[i]=T[0]
                    elif T[i]!=0 and T[0]==0:
                        T[0]=T[i]
                else:
                    if T[i+1]==0 and T[i]!=0:
                        T[i+1]=T[i]
                    elif T[i+1]!=0 and T[i]==0:
                        T[i]=T[i+1]
                    

            getallgasproperties(propslist,i+1)

            if processtype[i]=="isochoric" or processtype[i]=="Isochoric":
                if i==numberofstates-1:
                    if v[i]==0 and v[0]!=0:
                        v[i]=v[0]
                    elif v[i]!=0 and v[0]==0:
                        v[0]=v[i]
                    if wtransfer[i]=="" and qtransfer[i]=="" and v[i]!=0 and P[i]!=0 and P[0]!=0: #work, heat not given but all states fully defined.
                        vol=v[i]
                        wtransfer[i]=vol*(P[i]-P[0])
                    if wtransfer[i]!="" and qtransfer[i]=="" and P[0]==0 and P[i]!=0 and v[i]!=0: #use work transfer to find P
                        vol=v[i]
                        P[i]=((wtransfer[i])/vol)+P[0]
                    if wtransfer[i]!="" and qtransfer[i]=="" and P[0]!=0 and P[i]==0 and v[i]!=0:
                        vol=v[i]
                        P[0]=P[i]-((wtransfer[i])/vol)
                    if wtransfer[i]!="" and qtransfer=="" and P[0]!=0 and P[i]!=0 and v[i]==0:
                        v[i]=(wtransfer[i])/(P[i]-P[0])
                else:
                    if v[i+1]==0 and v[i]!=0:
                        v[i+1]=v[i]
                    elif v[i+1]!=0 and v[i]==0:
                        v[i]=v[i+1]
                    if wtransfer[i]=="" and qtransfer[i]=="" and v[i]!=0 and P[i]!=0 and P[i+1]!=0:
                        vol=v[i]
                        wtransfer[i]=vol*(P[i]-P[i+1])
                    if wtransfer[i]!="" and qtransfer[i]=="" and P[i+1]==0 and P[i]!=0 and v[i]!=0: #use work transfer to find P
                        vol=v[i]
                        P[i]=((wtransfer[i])/vol)+P[i+1]
                    if wtransfer[i]!="" and qtransfer[i]=="" and P[i+1]!=0 and P[i]==0 and v[i]!=0:
                        vol=v[i]
                        P[i+1]=P[i]-((wtransfer[i])/vol)
                    if wtransfer[i]!="" and qtransfer=="" and P[i+1]!=0 and P[i]!=0 and v[i]==0:
                        v[i]=(wtransfer[i])/(P[i]-P[i+1])

                

        
            getallgasproperties(propslist,i+1)

            if processtype[i]=="isenthalpic" or processtype[i]=="Isenthalpic":
                if i==numberofstates-1:
                    if h[i]==0 and h[0]!=0:
                        h[i]=h[0]
                    elif h[i]!=0 and h[0]==0:
                        h[0]=h[i]
                else:
                    if h[i+1]==0 and h[i]!=0:
                        h[i+1]=h[i]
                    elif h[i+1]!=0 and h[i]==0:
                        h[i]=h[i+1]

            getallgasproperties(propslist,i+1)

            if (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]>0 and isenefficiency[i]<1 and (componenttype[i]=="turbine" or componenttype[i]=="Turbine"): #i.e specified isentropic efficiency. 
                qtransfer[i]=0
                if i==numberofstates-1: #i.e process from i to 0
                    if h[0]==0 and h[i]!=0: #final unknown, initial known 
                        propslistcopy=copy.deepcopy(propslist)
                        propslistcopy[3][0]=propslistcopy[3][i] #ideal isentropic, hypothetical.
                        for j in range(numberofstates):
                            getallgasproperties(propslistcopy,j+1) #updating list
                        h_s=propslistcopy[4][0]
                        h[0]=(isenefficiency[i]*(h_s-h[i]))+h[i]
                    elif h[0]!=0 and h[i]==0: #final known, initial unknown 
                        propslistcopy=copy.deepcopy(propslist)
                        propslistcopy[3][i]=propslistcopy[3][0]
                        for j in range(numberofstates):
                            getallgasproperties(propslistcopy,j+1)
                        h_s=propslistcopy[4][0]
                        h[i]=((isenefficiency[i]*h_s)-h[0])/(isenefficiency[i]-1)
                elif i!=numberofstates-1:
                    if h[i+1]==0 and h[i]!=0: #final unknown, initial known 
                        propslistcopy=copy.deepcopy(propslist)
                        propslistcopy[3][i+1]=propslistcopy[3][i] #ideal isentropic, hypothetical.
                        for j in range(numberofstates):
                            getallgasproperties(propslistcopy,j+1) #updating list
                        h_s=propslistcopy[4][i+1]
                        h[i+1]=(isenefficiency[i]*(h_s-h[i]))+h[i]
                    elif h[i]==0 and h[i+1]!=0: #final known, initial unknown
                        propslistcopy=copy.deepcopy(propslist)
                        propslistcopy[3][i]=propslistcopy[3][i+1]
                        for j in range(numberofstates): 
                            getallgasproperties(propslistcopy,j+1)
                        h_s=propslistcopy[4][i+1]
                        h[i]=((isenefficiency[i]*h_s)-h[i+1])/(isenefficiency[i]-1)
                    
            if (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]>0 and isenefficiency[i]<1 and (componenttype[i]=="compressor" or componenttype[i]=="Compressor"):
                qtransfer[i]=0
                if i==numberofstates-1: #i.e process from i to 0
                    if h[0]==0 and h[i]!=0: #final unknown, initial known 
                        propslistcopy=copy.deepcopy(propslist)
                        propslistcopy[3][0]=propslistcopy[3][i] #ideal isentropic, hypothetical.
                        for j in range(numberofstates):
                            getallgasproperties(propslistcopy,j+1) #updating list
                        h_s=propslistcopy[4][0]
                        h[0]=((h_s-h[i])/isenefficiency[i])+h[i]
                    elif h[0]!=0 and h[i]==0: #final known, initial unknown 
                        propslistcopy=copy.deepcopy(propslist)
                        propslistcopy[3][i]=propslistcopy[3][0]
                        for j in range(numberofstates):
                            getallgasproperties(propslistcopy,j+1)
                        h_s=propslistcopy[4][0]
                        h[i]=(h_s-((isenefficiency[i])*h[0]))/(1-isenefficiency[i])
                else:
                    if h[i+1]==0 and h[i]!=0: #final unknown, initial known
                        propslistcopy=copy.deepcopy(propslist)
                        propslistcopy[3][i+1]=propslistcopy[3][i] #ideal isentropic, hypothetical.
                        for j in range(numberofstates):
                            getallgasproperties(propslistcopy,j+1)
                        h_s=propslistcopy[4][i+1]
                        h[i+1]=((h_s-h[i])/isenefficiency[i])+h[i]
                        
                    elif h[i]==0 and h[i+1]!=0: #final known, initial unknown
                        propslistcopy=copy.deepcopy(propslist)
                        propslistcopy[3][i]=propslistcopy[3][i+1]
                        for j in range(numberofstates):
                            getallgasproperties(propslistcopy,j+1)
                        h_s=propslistcopy[4][i+1]
                        h[i]=(h_s-((isenefficiency[i])*h[i+1]))/(1-isenefficiency[i])
            
            getallgasproperties(propslist,i+1)

            if (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]==1:
                qtransfer[i]=0
                if i==numberofstates-1:
                    if s[0]!=0 and s[i]==0:
                        s[i]=s[0]
                    elif s[0]==0 and s[i]!=0:
                        s[0]=s[i]
                else:
                    if s[i+1]!=0 and s[i]==0:
                        s[i]=s[i+1]
                    elif s[i]!=0 and s[i+1]==0:
                        s[i+1]=s[i]

            getallgasproperties(propslist,i+1)

            #next, using work transfer (wtransfer)
            for i in range(numberofstates):
                if componenttype[i]=="Compressor" or componenttype[i]=="compressor": #n_isen= w_isen/w_actual
                    if i==numberofstates-1:
                        if wtransfer[i]!="" and wtransfer[i]!=0 and checkfullydefidealgas(propslist,i+1)==True and checkfullydefidealgas(propslist,1)==False and isenefficiency[i]==0: #isentropic efficiency not defined. if it is defined and work is given, the situation is overdefined 
                            propslistcopy=copy.deepcopy(propslist) #trying to get ideal, isentropic work.
                            propslistcopy[3][0]=propslistcopy[3][i] #s2=s1
                            getallgasproperties(propslistcopy,1)
                            if h[0]==0:
                                h[0]=h[i]-wtransfer[i]
                            for k in range(numberofstates):
                                getallgasproperties(propslist,k+1)

                        elif wtransfer[i]!="" and wtransfer[i]!=0 and checkfullydefidealgas(propslist,1)==True and checkfullydefidealgas(propslist,i+1)==False and isenefficiency[i]==0: #isentropic efficiency not defined.
                            propslistcopy=copy.deepcopy(propslist)
                            propslistcopy[3][i]=propslistcopy[3][0] #s1=s2
                            getallgasproperties(propslistcopy,i+1)
                            if h[i]==0:
                                h[i]=h[0]+wtransfer[i]
                            for k in range(numberofstates):
                                getallgasproperties(propslist,k+1)

                    elif i!=numberofstates-1:
                        if wtransfer[i]!="" and wtransfer[i]!=0 and checkfullydefidealgas(propslist,i+1)==True and checkfullydefidealgas(propslist,i+2)==False and isenefficiency[i]==0: #isentropic efficiency not defined. if it is defined and work is given, the situation is overdefined 
                            propslistcopy=copy.deepcopy(propslist) #trying to get ideal, isentropic work.
                            propslistcopy[3][i+1]=propslistcopy[3][i] #s2=s1
                            getallgasproperties(propslistcopy,i+2)
                            if h[i+1]==0:
                                h[i+1]=h[i]-wtransfer[i]
                            for k in range(numberofstates):
                                getallgasproperties(propslist,k+1)

                        elif wtransfer[i]!="" and wtransfer[i]!=0 and checkfullydefidealgas(propslist,i+2)==True and checkfullydefidealgas(propslist,i+1)==False and isenefficiency[i]==0: #isentropic efficiency not defined.
                            propslistcopy=copy.deepcopy(propslist)
                            propslistcopy[3][i]=propslistcopy[3][i+1] #s1=s2
                            getallgasproperties(propslistcopy,i+1)
                            if h[i]==0:
                                h[i]=h[i+1]+wtransfer[i]
                            for k in range(numberofstates):
                                getallgasproperties(propslist,k+1)           
                
                elif componenttype[i]=="Turbine" or componenttype[i]=="turbine": #n_isen= w_actual/w_isen
                    if i==numberofstates-1:
                        if wtransfer[i]!="" and wtransfer[i]!=0 and checkfullydefidealgas(propslist,i+1)==True and checkfullydefidealgas(propslist,1)==False and isenefficiency[i]==0: #isentropic efficiency not defined. if it is defined and work is given, the situation is overdefined 
                            propslistcopy=copy.deepcopy(propslist) #trying to get ideal, isentropic work.
                            propslistcopy[3][0]=propslistcopy[3][i] #s2=s1
                            getallgasproperties(propslistcopy,1)
                            if h[0]==0:
                                h[0]=h[i]-wtransfer[i]
                            for k in range(numberofstates):
                                getallgasproperties(propslist,k+1)

                        elif wtransfer[i]!="" and wtransfer[i]!=0 and checkfullydefidealgas(propslist,1)==True and checkfullydefidealgas(propslist,i+1)==False and isenefficiency[i]==0: #isentropic efficiency not defined.
                            propslistcopy=copy.deepcopy(propslist)
                            propslistcopy[3][i]=propslistcopy[3][0] #s1=s2
                            getallgasproperties(propslistcopy,i+1)
                            if h[i]==0:
                                h[i]=h[0]+wtransfer[i]
                            for k in range(numberofstates):
                                getallgasproperties(propslist,k+1)


                    elif i!=numberofstates-1:
                        if wtransfer[i]!="" and wtransfer[i]!=0 and checkfullydefidealgas(propslist,i+1)==True and checkfullydefidealgas(propslist,i+2)==False and isenefficiency[i]==0: #isentropic efficiency not defined. if it is defined and work is given, the situation is overdefined 
                            propslistcopy=copy.deepcopy(propslist) #trying to get ideal, isentropic work.
                            propslistcopy[3][i+1]=propslistcopy[3][i] #s2=s1
                            getallgasproperties(propslistcopy,i+2)
                            if h[i+1]==0:
                                h[i+1]=h[i]-wtransfer[i]
                            for k in range(numberofstates):
                                getallgasproperties(propslist,k+1)

                        elif wtransfer[i]!="" and wtransfer[i]!=0 and checkfullydefidealgas(propslist,i+2)==True and checkfullydefidealgas(propslist,i+1)==False and isenefficiency[i]==0: #isentropic efficiency not defined.
                            propslistcopy=copy.deepcopy(propslist)
                            propslistcopy[3][i]=propslistcopy[3][i+1] #s1=s2
                            getallgasproperties(propslistcopy,i+1)
                            if h[i]==0:
                                h[i]=h[i+1]+wtransfer[i]
                            for k in range(numberofstates):
                                getallgasproperties(propslist,k+1)             


            getallgasproperties(propslist,i+1)

            if i==numberofstates-1:
                if qtransfer[i]!="" and wtransfer[i]!="" and h[i]!=0 and h[0]==0: #q,w and h1 known, h2 UNKNOWN
                    h[0]=qtransfer[i]-wtransfer[i]+h[i]+((c[i]**2)/2)+(9.81*z[i])-((c[0]**2)/2)-(9.81*z[0])
                elif qtransfer[i]!="" and wtransfer[i]!="" and h[0]!=0 and h[i]==0: #q, w and h2 known. h1 unknown. 
                    h[i]=h[0]+((c[0]**2)/2)+(9.81*z[0])-((c[i]**2)/2)-(9.81*z[i])+wtransfer[i]-qtransfer[i] 
                elif qtransfer[i]!="" and h[i]!=0 and h[0]!=0 and wtransfer[i]=="": #q, h1, h2 known. w UNKNOWN
                    wtransfer[i]=qtransfer[i]+h[i]-h[0]+((c[i]**2)/2)+(9.81*z[i])-((c[0]**2)/2)-(9.81*z[0])
                elif wtransfer[i]!="" and h[i]!=0 and h[0]!=0 and qtransfer[i]=="": #w, h1, h2 known. q UNKNOWN
                    qtransfer[i]=h[0]-h[i]+((c[0]**2)/2)+(9.81*z[0])-((c[i]**2)/2)-(9.81*z[i])+wtransfer[i]
            else:
                if qtransfer[i]!="" and wtransfer[i]!="" and h[i]!=0 and h[i+1]==0: #q,w and h1 known, h2 UNKNOWN
                    h[i+1]=qtransfer[i]-wtransfer[i]+h[i]+((c[i]**2)/2)+(9.81*z[i])-((c[i+1]**2)/2)-(9.81*z[i+1])
                elif qtransfer[i]!="" and wtransfer[i]!="" and h[i+1]!=0 and h[i]==0: #q, w and h2 known. h1 unknown. 
                    h[i]=h[i+1]+((c[i+1]**2)/2)+(9.81*z[i+1])-((c[i]**2)/2)-(9.81*z[i])+wtransfer[i]-qtransfer[i] 
                elif qtransfer[i]!="" and h[i]!=0 and h[i+1]!=0 and wtransfer[i]=="": #q, h1, h2 known. w UNKNOWN
                    wtransfer[i]=qtransfer[i]+h[i]-h[i+1]+((c[i]**2)/2)+(9.81*z[i])-((c[i+1]**2)/2)-(9.81*z[i+1])
                elif wtransfer[i]!="" and h[i]!=0 and h[i+1]!=0 and qtransfer[i]=="": #w, h1, h2 known. q UNKNOWN
                    qtransfer[i]=h[i+1]-h[i]+((c[i+1]**2)/2)+(9.81*z[i+1])-((c[i]**2)/2)-(9.81*z[i])+wtransfer[i]
            
            getallgasproperties(propslist,i+1)

            for i in range(numberofstates):
                if qtransfer[i]!="" and Qdot[i]!="" and qtransfer[i]!=0 and Qdot[1]!=0 and massflowrate==0: #i.e massflowrate not given, must be solved for.
                    massflowrate=(Qdot[i])/(qtransfer[i])
                if wtransfer[i]!="" and Wdot[i]!="" and wtransfer[i]!=0 and Wdot[1]!=0 and massflowrate==0: #i.e massflowrate not given, must be solved for.
                    massflowrate=(Wdot[i])/(wtransfer[i])

            if qtransfer[i]=="" and Qdot[i]!="" and massflowrate!=0:
                qtransfer[i]=(Qdot[i])/massflowrate
            
            if Qdot[i]=="" and qtransfer[i]!="" and massflowrate!=0:
                Qdot[i]=massflowrate*qtransfer[i]
            
            if wtransfer[i]=="" and Wdot[i]!="" and massflowrate!=0:
                wtransfer[i]=(Wdot[i])/massflowrate
            
            if Wdot[i]=="" and wtransfer[i]!="" and massflowrate!=0:
                Wdot[i]=massflowrate*wtransfer[i]
            #makes sure that Qdot/Wdot lists are complying with q and w lists.

        for i in range(0,numberofstates):
            getallgasproperties(propslist,i+1)
        #use of SFEE:
        #note: SFEE can only solve for q, w, h2, h1. does NOT solve for c and/ or z.
        #BASICALLY, THREE OF THE QTYS OUT OF q,w,h2,h1 MUST be known. 
        for i in range(0,numberofstates):
            if i==numberofstates-1:
                if qtransfer[i]!="" and wtransfer[i]!="" and h[i]!=0 and h[0]==0: #q,w and h1 known, h2 UNKNOWN
                    h[0]=qtransfer[i]-wtransfer[i]+h[i]+((c[i]**2)/2)+(9.81*z[i])-((c[0]**2)/2)-(9.81*z[0])
                elif qtransfer[i]!="" and wtransfer[i]!="" and h[0]!=0 and h[i]==0: #q, w and h2 known. h1 unknown. 
                    h[i]=h[0]+((c[0]**2)/2)+(9.81*z[0])-((c[i]**2)/2)-(9.81*z[i])+wtransfer[i]-qtransfer[i] 
                elif qtransfer[i]!="" and h[i]!=0 and h[0]!=0 and wtransfer[i]=="": #q, h1, h2 known. w UNKNOWN
                    wtransfer[i]=qtransfer[i]+h[i]-h[0]+((c[i]**2)/2)+(9.81*z[i])-((c[0]**2)/2)-(9.81*z[0])
                elif wtransfer[i]!="" and h[i]!=0 and h[0]!=0 and qtransfer[i]=="": #w, h1, h2 known. q UNKNOWN
                    qtransfer[i]=h[0]-h[i]+((c[0]**2)/2)+(9.81*z[0])-((c[i]**2)/2)-(9.81*z[i])+wtransfer[i]
            else:
                if qtransfer[i]!="" and wtransfer[i]!="" and h[i]!=0 and h[i+1]==0: #q,w and h1 known, h2 UNKNOWN
                    h[i+1]=qtransfer[i]-wtransfer[i]+h[i]+((c[i]**2)/2)+(9.81*z[i])-((c[i+1]**2)/2)-(9.81*z[i+1])
                elif qtransfer[i]!="" and wtransfer[i]!="" and h[i+1]!=0 and h[i]==0: #q, w and h2 known. h1 unknown. 
                    h[i]=h[i+1]+((c[i+1]**2)/2)+(9.81*z[i+1])-((c[i]**2)/2)-(9.81*z[i])+wtransfer[i]-qtransfer[i] 
                elif qtransfer[i]!="" and h[i]!=0 and h[i+1]!=0 and wtransfer[i]=="": #q, h1, h2 known. w UNKNOWN
                    wtransfer[i]=qtransfer[i]+h[i]-h[i+1]+((c[i]**2)/2)+(9.81*z[i])-((c[i+1]**2)/2)-(9.81*z[i+1])
                elif wtransfer[i]!="" and h[i]!=0 and h[i+1]!=0 and qtransfer[i]=="": #w, h1, h2 known. q UNKNOWN
                    qtransfer[i]=h[i+1]-h[i]+((c[i+1]**2)/2)+(9.81*z[i+1])-((c[i]**2)/2)-(9.81*z[i])+wtransfer[i]
            getallgasproperties(propslist,i+1)

        for i in range(0,numberofstates):
            getallgasproperties(propslist,i+1)

        #solving for mdot(possibly)
        for i in range(numberofstates):
            if qtransfer[i]!="" and Qdot[i]!="" and qtransfer[i]!=0 and Qdot[1]!=0 and massflowrate==0: #i.e massflowrate not given, must be solved for.
                massflowrate=(Qdot[i])/(qtransfer[i])
            if wtransfer[i]!="" and Wdot[i]!="" and wtransfer[i]!=0 and Wdot[1]!=0 and massflowrate==0: #i.e massflowrate not given, must be solved for.
                massflowrate=(Wdot[i])/(wtransfer[i])
            
        for i in range(0,numberofstates):
            getallgasproperties(propslist,i+1)




    for i in range(numberofstates):
        if qtransfer[i]=="" and Qdot[i]!="" and massflowrate!=0:
            qtransfer[i]=(Qdot[i])/massflowrate
            
        if Qdot[i]=="" and qtransfer[i]!="" and massflowrate!=0:
            Qdot[i]=massflowrate*qtransfer[i]
            
        if wtransfer[i]=="" and Wdot[i]!="" and massflowrate!=0:
            wtransfer[i]=(Wdot[i])/massflowrate
            
        if Wdot[i]=="" and wtransfer[i]!="" and massflowrate!=0:
            Wdot[i]=massflowrate*wtransfer[i]

    #done once more

    for i in range(0,numberofstates):
            getallgasproperties(propslist,i+1) #updating the list again.

    #calculating isen efficiencies if not entered, ASSUMING ALL STATES FULLY DEFINED. OUT OF THE LOOP.
    w_isen=0
    w_actual=0
    for i in range(numberofstates):
        if (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]==0:
            if i==numberofstates-1:
                propslistclone=copy.deepcopy(propslist)
    #setting all properties EXCEPT PRESSURE to default, for FINAL STATE.
                for k in range(1,6):
                    propslistclone[k][0]=0
                propslistclone[3][0]=propslistclone[3][i]#ideal isentropic case. 
                getallgasproperties(propslistclone,1)
                w_isen=propslistclone[4][i]-propslistclone[4][0]
                w_actual=propslist[4][i]-propslist[4][0]
                if componenttype[i]=="compressor" or componenttype[i]=="Compressor":
                    isenefficiency[i]=w_isen/w_actual
                elif componenttype[i]=="turbine" or componenttype[i]=="Turbine":
                    isenefficiency[i]=w_actual/w_isen

            elif i!=numberofstates-1:
                propslistclone=copy.deepcopy(propslist)
                for k in range(1,6):
                    propslistclone[k][i+1]=0
                propslistclone[3][i+1]=propslistclone[3][i]#ideal isentropic case. 
                getallgasproperties(propslistclone,i+2)
                w_isen=propslistclone[4][i]-propslistclone[4][i+1]
                w_actual=propslist[4][i]-propslist[4][i+1]
                if componenttype[i]=="compressor" or componenttype[i]=="Compressor":
                    isenefficiency[i]=w_isen/w_actual
                elif componenttype[i]=="turbine" or componenttype[i]=="Turbine":
                    isenefficiency[i]=w_actual/w_isen

    for i in range(numberofstates):
        if qtransfer[i]!="" and Qdot[i]!="" and qtransfer[i]!=0 and Qdot[1]!=0 and massflowrate==0: #i.e massflowrate not given, must be solved for.
            massflowrate=(Qdot[i])/(qtransfer[i])
        if wtransfer[i]!="" and Wdot[i]!="" and wtransfer[i]!=0 and Wdot[1]!=0 and massflowrate==0: #i.e massflowrate not given, must be solved for.
            massflowrate=(Wdot[i])/(wtransfer[i])

    for i in range(numberofstates):
        if qtransfer[i]=="" and Qdot[i]!="" and massflowrate!=0:
            qtransfer[i]=(Qdot[i])/massflowrate
            
        if Qdot[i]=="" and qtransfer[i]!="" and massflowrate!=0:
            Qdot[i]=massflowrate*qtransfer[i]
            
        if wtransfer[i]=="" and Wdot[i]!="" and massflowrate!=0:
            wtransfer[i]=(Wdot[i])/massflowrate
            
        if Wdot[i]=="" and wtransfer[i]!="" and massflowrate!=0:
            Wdot[i]=massflowrate*wtransfer[i]


    #plotting the graphs, assuming that all states have been fully defined.
    #----------------------------------------------------------------------------------------------------------#
    #1 Pv diag
    plotpv = figure(title="P-v plot", x_axis_label="v (m^3/kg)", y_axis_label="P (Pa)", width=540, height=540)
    plist=[]
    vlist=[] #just for assignment
    a=0
    b=0
    comptype=""
    for i in range(numberofstates):
        index=i
        if componenttype[i]=="boiler"  or componenttype[i]=="Boiler":
            comptype="boiler"
        if componenttype[i]=="condensor"  or componenttype[i]=="Condensor":
            comptype="condensor"
        if componenttype[i]=="compressor"  or componenttype[i]=="Compressor":
            comptype="compressor"
        if componenttype[i]=="turbine"  or componenttype[i]=="Turbine":
            comptype="turbine"
        if componenttype[i]=="throttle" or componenttype[i]=="Throttle":
            comptype="throttle"
        if componenttype[i]=="":
            comptype="undefined"
        
        if processtype[i]=="isobaric" or processtype[i]=="Isobaric":
            if i==numberofstates-1:
                pressure=P[i] #which must, hence, be equal lto P[0]
                v_in=v[i]
                v_fin=v[0]
                vlist=np.linspace(v_in,v_fin) #final as well as initial is included
                plist=[]
                for j in vlist:
                    plist+=[pressure]
            else:
                pressure=P[i] #which must, hence, be equal lto P[i+1]
                v_in=v[i]
                v_fin=v[i+1]
                vlist=np.linspace(v_in,v_fin) #final as well as initial is included
                plist=[]
                for j in vlist:
                    plist+=[pressure]
        elif processtype[i]=="isochoric" or processtype[i]=="Isochoric":
            if i==numberofstates-1:
                specvol=v[i] #this must also be equal to v[0]
                P_in=P[i]
                P_fin=P[0]
                plist=np.linspace(P_in,P_fin)
                vlist=[]
                for i in plist:
                    vlist+=[specvol]
            else:
                specvol=v[i] #this must also be equal to v[0]
                P_in=P[i]
                P_fin=P[i+1]
                plist=np.linspace(P_in,P_fin)
                vlist=[]
                for i in plist:
                    vlist+=[specvol]
        elif processtype[i]=="isothermal" or processtype[i]=="Isothermal":
            if i==numberofstates-1:
                Temperature=T[i]
                v_in=v[i]
                v_fin=v[0]
                vlist=np.linspace(v_in,v_fin)
                plist=[]
                for i in vlist: 
                    plist+=[CP.PropsSI("P","D",1/i,"T",Temperature,fluid)]
            else: 
                Temperature=T[i]
                v_in=v[i]
                v_fin=v[i+1]
                vlist=np.linspace(v_in,v_fin)
                plist=[]
                for i in vlist:
                    plist+=[CP.PropsSI("P","D",1/i,"T",Temperature,fluid)]

        elif processtype[i]=="isenthalpic" or processtype[i]=="Isenthalpic":
            if i==numberofstates-1:
                enthalpy=h[i]
                P_in=P[i]
                P_fin=P[0]
                plist=np.linspace(P_in,P_fin)
                vlist=[]
                for i in plist:
                    vlist+=[1/(CP.PropsSi("D","P",i,"H",enthalpy,fluid))]
            else:
                enthalpy=h[i]
                P_in=P[i]
                P_fin=P[i+1]
                plist=np.linspace(P_in,P_fin)
                vlist=[]
                for i in plist:
                    vlist+=[1/(CP.PropsSI("D","P",i,"H",enthalpy,fluid))]
        
        elif (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]!=1 and isenefficiency[i]!=0:#isenefficiency[i]>0 and isenefficiency[i]<1: #adiabatic and irreversible, using approximation p=(a/v)+b
            if i==numberofstates-1:
                P1=P[i]
                v1=v[i]
                P2=P[0]
                v2=v[0]
                b=((P1*v1)-(P2*v2))/(v1-v2)
                a=(P1*v1)-(b*v1)
                vlist=np.linspace(v1,v2)
                plist=[]
                for i in vlist:
                    plist+=[(a/i)+b]
            else:
                P1=P[i]
                v1=v[i]
                P2=P[i+1]
                v2=v[i+1]
                b=((P1*v1)-(P2*v2))/(v1-v2)
                a=(P1*v1)-(b*v1)
                vlist=np.linspace(v1,v2)
                plist=[]
                for i in vlist:
                    plist+=[(a/i)+b]        

        elif (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]==1.0: #adiabatic and reversible, i.e isentropic. 
            if i==numberofstates-1:
                entropy=s[i] #same as s[0]
                P_in=P[i]
                P_fin=P[0]
                plist=np.linspace(P_in,P_fin)
                vlist=[]
                for i in plist:
                    vlist+=[1/(CP.PropsSI("D","P",i,"S",entropy,fluid))]
            else:
                entropy=s[i]
                P_in=P[i]
                P_fin=P[i+1]
                plist=np.linspace(P_in,P_fin)
                vlist=[]
                for j in plist:
                    vlist+=[1/(CP.PropsSI("D","P",j,"S",entropy,fluid))]

        if comptype=="boiler":
            plotpv.line(vlist,plist,line_width=2,color="orange",legend_label="Component "+str(index)+": Boiler")
        elif comptype=="condensor":
            plotpv.line(vlist,plist,line_width=2,color="red",legend_label="Component "+str(index)+": Condensor")
        elif comptype=="compressor":
            plotpv.line(vlist,plist,line_width=2,color="green",legend_label="Component "+str(index)+": Compressor")
        elif comptype=="turbine":
            plotpv.line(vlist,plist,line_width=2,color="blue",legend_label="Component "+str(index)+": Turbine")
        elif comptype=="throttle":
            plotpv.line(vlist,plist,line_width=2,color="purple",legend_label="Component "+str(index)+": Throttle")
        elif comptype=="undefined":
            plotpv.line(vlist,plist,line_width=2,color="brown",legend_label="Component "+str(index)+": Undefined")
    plotpv.legend.location="top_right"
    plotpv.legend.title="Legend (for all 3 plots)"
    plotpv.legend.label_text_font_size="9pt"
    plotpv.legend.title_text_font_size="9pt"
    labels=[]
    for i in range(numberofstates):
        labels+=["State "+str(i)]



    #---------------------------------------------------------------------------------------------------------#
    #now doing ts (note: non isentropic case not done rn)
    plotts = figure(title="T-s plot", x_axis_label="s (J/kgK)", y_axis_label="T (K)", width=540, height=540)
    Tlist=[]
    slist=[] #just for assignment. 
    comptype=""
    for i in range(numberofstates):
        if componenttype[i]=="boiler"  or componenttype[i]=="Boiler":
            comptype="boiler"
        if componenttype[i]=="condensor"  or componenttype[i]=="Condensor":
            comptype="condensor"
        if componenttype[i]=="compressor"  or componenttype[i]=="Compressor":
            comptype="compressor"
        if componenttype[i]=="turbine"  or componenttype[i]=="Turbine":
            comptype="turbine"
        if componenttype[i]=="Throttle" or componenttype[i]=="throttle":
            comptype="throttle"
        if componenttype[i]=="":
            comptype="undefined"
        
        if processtype[i]=="isobaric" or processtype[i]=="Isobaric":
            if i==numberofstates-1:
                pressure=P[i] #which must, hence, be equal lto P[0]
                s_in=s[i]
                s_fin=s[0]
                slist=np.linspace(s_in,s_fin)
                Tlist=[]
                for i in slist:
                    Tlist+=[CP.PropsSI("T","S",i,"P",pressure,fluid)]
            else:
                pressure=P[i] #which must, hence, be equal lto P[0]
                s_in=s[i]
                s_fin=s[i+1]
                slist=np.linspace(s_in,s_fin)
                Tlist=[]
                for i in slist:
                    Tlist+=[CP.PropsSI("T","S",i,"P",pressure,fluid)]
                    
        elif processtype[i]=="isochoric" or processtype[i]=="Isochoric":
            if i==numberofstates-1:
                specvol=v[i] #this must also be equal to v[0]
                T_in=T[i]
                T_fin=T[0]
                Tlist=np.linspace(T_in,T_fin)
                slist=[]
                for i in Tlist:
                    slist+=[CP.PropsSI("S","D",1/specvol,"T",i,fluid)]
            else:
                specvol=v[i] #this must also be equal to v[0]
                T_in=T[i]
                T_fin=T[i+1]
                Tlist=np.linspace(T_in,T_fin)
                slist=[]
                for i in Tlist:
                    slist+=[CP.PropsSI("S","D",1/specvol,"T",i,fluid)]

        elif processtype[i]=="isothermal" or processtype[i]=="Isothermal":
            if i==numberofstates-1:
                Temperature=T[i]
                s_in=s[i]
                s_fin=s[0]
                Tlist=[]
                slist=np.linspace(s_in,s_fin)
                for i in slist:
                    Tlist+=[Temperature]
            else: 
                Temperature=T[i]
                s_in=s[i]
                s_fin=s[i+1]
                Tlist=[]
                slist=np.linspace(s_in,s_fin)
                for i in slist:
                    Tlist+=[Temperature]

        elif processtype[i]=="isenthalpic" or processtype[i]=="Isenthalpic":
            if i==numberofstates-1:
                enthalpy=h[i]
                s_in=s[i]
                s_fin=s[0]
                slist=np.linspace(s_in,s_fin)
                Tlist=[]
                for i in slist:
                    Tlist+=[CP.PropsSI("T","S",i,"H",enthalpy,fluid)]

            else:
                enthalpy=h[i]
                s_in=s[i]
                s_fin=s[i+1]
                slist=np.linspace(s_in,s_fin)
                Tlist=[]
                for i in slist:
                    Tlist+=[CP.PropsSI("T","S",i,"H",enthalpy,fluid)]
        
        #elif processtype[i]=="adiabatic" or processtype[i]=="Adiabatic" and isenefficiency[i]>0 and isenefficiency[i]<1: #adiabatic and irreversible, NOT SURE how this should appear in P-v diagram. 

        elif (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]==1.0: #adiabatic and reversible, i.e isentropic, constant entropy.  
            if i==numberofstates-1:
                entropy=s[i] #same as s[0]
                T_in=T[i]
                T_fin=T[0]
                Tlist=np.linspace(T_in,T_fin)
                slist=[]
                for i in Tlist:
                    slist+=[entropy]

            else:
                entropy=s[i] #same as s[0]
                T_in=T[i]
                T_fin=T[i+1]
                Tlist=np.linspace(T_in,T_fin)
                slist=[]
                for i in Tlist:
                    slist+=[entropy]
        elif (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]!=1 and isenefficiency[i]!=0:#isenefficiency[i]>0 and isenefficiency[i]<1: #adiabatic, irreversible. plotted with straight line for ts and hs.
            if i==numberofstates-1:
                s_in=s[i]
                s_fin=s[0]
                slist=[s_in,s_fin]
                T_in=T[i]
                T_fin=T[0]
                Tlist=[T_in,T_fin]
            elif i!=numberofstates-1:
                s_in=s[i]
                s_fin=s[i+1]
                slist=[s_in,s_fin]
                T_in=T[i]
                T_fin=T[i+1]
                Tlist=[T_in,T_fin]

        if comptype=="boiler":
            plotts.line(slist,Tlist,line_width=2,color="orange")
        elif comptype=="condensor":
            plotts.line(slist,Tlist,line_width=2,color="red")
        elif comptype=="compressor":
            plotts.line(slist,Tlist,line_width=2,color="green")
        elif comptype=="turbine":
            plotts.line(slist,Tlist,line_width=2,color="blue")
        elif comptype=="throttle":
            plotts.line(slist,Tlist,line_width=2,color="purple")
        elif comptype=="undefined":
            plotts.line(slist,Tlist,line_width=2,color="brown")
    #--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
    #now doing hs
    plot_hs = figure(title="h-s plot", x_axis_label="s (J/kgK)", y_axis_label="h (J/kg)", width=540, height=540)
    hlist1=[]
    slist1=[]
    comptype=""
    for i in range(numberofstates):
        if componenttype[i]=="boiler"  or componenttype[i]=="Boiler":
            comptype="boiler"
        if componenttype[i]=="condensor"  or componenttype[i]=="Condensor":
            comptype="condensor"
        if componenttype[i]=="compressor"  or componenttype[i]=="Compressor":
            comptype="compressor"
        if componenttype[i]=="turbine"  or componenttype[i]=="Turbine":
            comptype="turbine"
        if componenttype[i]=="Throttle" or componenttype[i]=="throttle":
            comptype="throttle"
        if componenttype[i]=="":
            comptype="undefined"
            
        if processtype[i]=="isobaric" or processtype[i]=="Isobaric":
            if i==numberofstates-1:
                pressure=P[i] #which must, hence, be equal lto P[0]
                s_in=s[i]
                s_fin=s[0]
                slist1=np.linspace(s_in,s_fin)
                hlist1=[]
                for i in slist1:
                    hlist1+=[CP.PropsSI("H","S",i,"P",pressure,fluid)]
            else:
                pressure=P[i] #which must, hence, be equal lto P[0]
                s_in=s[i]
                s_fin=s[i+1]
                slist1=np.linspace(s_in,s_fin)
                hlist1=[]
                for i in slist1:
                    hlist1+=[CP.PropsSI("H","S",i,"P",pressure,fluid)]
                    
        elif processtype[i]=="isochoric" or processtype[i]=="Isochoric":
            if i==numberofstates-1:
                specvol=v[i] #this must also be equal to v[0]
                s_in=s[i]
                s_fin=s[0]
                slist1=np.linspace(s_in,s_fin)
                hlist1=[]
                for i in slist1:
                    hlist1+=[CP.PropsSI("H","D",1/specvol,"S",i,fluid)]
            else:
                specvol=v[i] #this must also be equal to v[0]
                s_in=s[i]
                s_fin=s[i+1]
                slist1=np.linspace(s_in,s_fin)
                hlist1=[]
                for i in slist1:
                    hlist1+=[CP.PropsSI("H","D",1/specvol,"S",i,fluid)]

        elif processtype[i]=="isothermal" or processtype[i]=="Isothermal":
            if i==numberofstates-1:
                Temperature=T[i]
                s_in=s[i]
                s_fin=s[0]
                slist1=np.linspace(s_in,s_fin)
                hlist1=[]
                for i in slist1:
                    hlist1+=[CP.PropsSI("H","T",Temperature,"S",i,fluid)]
            else: 
                Temperature=T[i]
                s_in=s[i]
                s_fin=s[i+1]
                slist1=np.linspace(s_in,s_fin)
                hlist1=[]
                for i in slist1:
                    hlist1+=[CP.PropsSI("H","T",Temperature,"S",i,fluid)]

        elif processtype[i]=="isenthalpic" or processtype[i]=="Isenthalpic":
            if i==numberofstates-1:
                enthalpy=h[i]
                s_in=s[i]
                s_fin=s[0]
                slist1=np.linspace(s_in,s_fin)
                hlist1=[]
                for i in slist1:
                    hlist1+=[enthalpy]

            else:
                enthalpy=h[i]
                s_in=s[i]
                s_fin=s[i+1]
                slist1=np.linspace(s_in,s_fin)
                hlist1=[]
                for i in slist1:
                    hlist1+=[enthalpy]
        

        elif (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]==1.0: #adiabatic and reversible, i.e isentropic, constant entropy.  
            if i==numberofstates-1:
                entropy=s[i] #same as s[0]
                h_in=h[i]
                h_fin=h[0]
                hlist1=np.linspace(h_in,h_fin)
                slist1=[]
                for i in hlist1:
                    slist1+=[entropy]

            else:
                entropy=s[i] #same as s[0]
                h_in=h[i]
                h_fin=h[i+1]
                hlist1=np.linspace(h_in,h_fin)
                slist1=[]
                for i in hlist1:
                    slist1+=[entropy]
        elif (processtype[i]=="adiabatic" or processtype[i]=="Adiabatic") and isenefficiency[i]!=1 and isenefficiency[i]!=0:#isenefficiency[i]>0 and isenefficiency[i]<1: #adiabatic, irreversible. plotted with straight line for ts and hs.
            if i==numberofstates-1:
                s_in=s[i]
                s_fin=s[0]
                slist1=[s_in,s_fin]
                h_in=h[i]
                h_fin=h[0]
                hlist1=[h_in,h_fin]
            elif i!=numberofstates-1:
                s_in=s[i]
                s_fin=s[i+1]
                slist1=[s_in,s_fin]
                h_in=h[i]
                h_fin=h[i+1]
                hlist1=[h_in,h_fin]

        if comptype=="boiler":
            plot_hs.line(slist1,hlist1,line_width=2,color="orange")
        elif comptype=="condensor":
            plot_hs.line(slist1,hlist1,line_width=2,color="red")
        elif comptype=="compressor":
            plot_hs.line(slist1,hlist1,line_width=2,color="green")
        elif comptype=="turbine":
            plot_hs.line(slist1,hlist1,line_width=2,color="blue")
        elif comptype=="throttle":
            plot_hs.line(slist1,hlist1,line_width=2,color="purple")
        elif comptype=="undefined":
            plot_hs.line(slist1,hlist1,line_width=2,color="brown")

    #calculating efficiency 
    netspecwork=0
    heatinput=0
    for i in wtransfer:
        netspecwork+=float(i)
    for j in qtransfer:
        if float(j)>0:
            heatinput+=float(j)
        
    if heatinput>0:
        efficiency=(netspecwork/heatinput)*100

    #--------------------------------------------#
    #NOW CREATING THE PROPERTIES TABLE:
    data = dict(
        Press=P,
        specvol=v,
        inten=u,
        ent=s,
        temp=T,
        enthal=h
    )

    source = ColumnDataSource(data=data)

    # Create columns for the table
    columns = [
        TableColumn(field='temp', title='T(K)'),
        TableColumn(field='Press', title='P(Pa)'),
        TableColumn(field='specvol', title='v(m^3/kg)'),
        TableColumn(field='inten', title='u(J/kg)'),
        TableColumn(field='enthal', title='h(J/kg)'),
        TableColumn(field='ent', title='s(J/kgK)'),
    ]
    twodpformat=NumberFormatter(format="0.00")

    for i in range(len(columns)):
        columns[i].formatter=twodpformat

    # Create the DataTable
    table = DataTable(source=source, columns=columns, editable=False,height=150)

    #---------------------------------------------------------------------------------------------------------------#

    #NOW CREATING THE HEAT AND WORK TABLE (SPECIFIC ONLY, FOR NOW):


    if solveforspecific!=False:
        data2 = dict(
            heat=qtransfer,
            work=wtransfer,
            processtype=processtype
        )

        source2 = ColumnDataSource(data=data2)

        # Create columns for the table
        columns2 = [
            TableColumn(field="heat", title="q(J/kg)"),
            TableColumn(field="work",title="w(J/kg)"),
            TableColumn(field="processtype", title="Process Type")
        ]

        twodpformat=NumberFormatter(format="0.00")

        for i in range(len(columns2)-1):
            columns2[i].formatter=twodpformat
        
        #efficiency display
        efficiencyrounded="%.2f" % efficiency
        efficiency_div = Div(text="Thermal efficiency= "+str(efficiencyrounded)+"%", width=300)

        mflowraterounded="%.2f" % massflowrate
        mflowrate_div=Div(text="Mass flow rate= "+str(mflowraterounded)+ " kg/s",width=300)

        # Create the DataTable
        table2 = DataTable(source=source2, columns=columns2, editable=False,height=150)



    elif solveforspecific==False:
        data2 = dict(
            heat=Qdot,
            work=Wdot,
            processtype=processtype
        )

        source2 = ColumnDataSource(data=data2)

        # Create columns for the table
        columns2 = [
            TableColumn(field="heat", title="Qdot (W)"),
            TableColumn(field="work",title="Wdot (W)"),
            TableColumn(field="processtype", title="Process Type")
        ]

        twodpformat=NumberFormatter(format="0.00")

        for i in range(len(columns2)-1):
            columns2[i].formatter=twodpformat
        
        #efficiency display
        efficiencyrounded="%.2f" % efficiency
        efficiency_div = Div(text="Thermal efficiency= "+str(efficiencyrounded)+"%", width=200)

        mflowraterounded="%.2f" % massflowrate
        mflowrate_div=Div(text="Mass flow rate= "+str(mflowraterounded)+ " kg/s",width=200)

        # Create the DataTable
        table2 = DataTable(source=source2, columns=columns2, editable=False,height=150)

    turb_iseneff=0
    comp_iseneff=0 #just for assignment
    for i in range(numberofstates):
        if componenttype[i]=="compressor" or componenttype[i]=="Compressor":
            comp_iseneff=(isenefficiency[i])*100
        if componenttype[i]=="Turbine" or componenttype[i]=="turbine":
            turb_iseneff=(isenefficiency[i])*100
    comp_div=Div(text="Compressor isentropic efficiency: "+str("%.2f" % comp_iseneff)+"%",width=300)
    turb_div=Div(text="Turbine isentropic efficiency: "+str("%.2f" % turb_iseneff)+"%",width=300)
    note_div=Div(text="Note: the lines representing adiabatic irreversible processes are approximated fits.",width=600)

    isenwrong_div=Div(text="")
    effneg_div=Div(text="")
    for i in range(numberofstates):
        if isenefficiency[i]<0 or isenefficiency[i]>1:
            isenwrong_div=Div(text="Check entered values: isentropic efficiency must be between 0% & 100%.",style={'color': 'blue'})
    if efficiency<0:
        effneg_div=Div(text="Negative thermal efficiency means there is no work producing device present.",style={'color': 'purple'})

    disclaimerdiv=Div(text="Values computed by the ThermoSolver program for gases may differ from those calculated by hand, because property table data has been used in this program, causing deviation from the usually assumed ideal-perfect nature of gases.",style={'color': 'red'})

    # Create a layout that combines the table and the plot
    layout = column(row(plotts,plot_hs,plotpv),row(table,table2,column(isenwrong_div,effneg_div)),row(column(efficiency_div,mflowrate_div),column(comp_div,turb_div),column(note_div,disclaimerdiv)))
    show(layout)

elif fluid=="water":
    errorDiv=Div(text="Error: This code only works for ideal perfect gases. Change the fluid from the Initialiser.ipynb file, or run the file launch_program.ipynb instead.",width=1200)
    show(errorDiv)